## About this Example

This notebook contains sample code for switching data sources without changing code. We use **environment variables** to determine which data source to use.

Notebook UI in *Watson Studio* does not have the capability to set environment variables, but Jobs UI for notebooks provides this capability. Because of the current design:
1. We set the environment variable programmatically, test the notebook, and then comment out the code.
2. When we schedule a notebook job, we set the environment variable. Since we commented out the code in Step 1, only the environment variable will be used to determine which connection to use.

In this example we are using two DB connections that have been defined in the project (*DB2_Dev* and *DB2_Prod*) that have different schemas but the same table. You can modify the code for your scenario, for example switching between database and file data sources. 

## Dev data source: test

In [ ]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

DB2_Dev_data_request = {
    'connection_name': """DB2_Dev""",
    'select_statement': 'SELECT * FROM "JLD84201"."CUST_TRANS"'
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=DB2_Dev_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo)
data_df_1.head(10)


## Prod data source: test

In [ ]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

DB2_Prod_data_request = {
    'connection_name': """DB2_Prod""",
    'select_statement': 'SELECT * FROM "TVV29432"."CUST_TRANS"'
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=DB2_Prod_data_request)

data_df_2 = itcfs.read_pandas_and_concat(readClient, flightInfo)
data_df_2.head(10)


## Switch Schema Names based on env var: for testing only

In [ ]:
import os

# Comment out the set env code before scheduling it as a Job
# Set schema name for testing this notebook interactively

# Dev schema
#os.environ['SCHEMA_NAME'] = 'JLD84201'
# Prod schema
#os.environ['SCHEMA_NAME'] = 'TVV29432'

In [ ]:
%env

In [ ]:
# Determine if this notebook needs to connect to a dev or a prod schema
# Schema name will be set in a notebook job definition

# Let's check if env variable was set
schema_name=os.getenv('SCHEMA_NAME')

# If it wasn't set, the default schema is dev
if schema_name is None:
  schema_name = 'JLD84201'

print("Schema: " + schema_name)

In [ ]:
type(schema_name)

In [ ]:
# Construct select string with schema_name variable
sqlSelect = 'SELECT * FROM ' + '"' + schema_name + '"."CUST_TRANS"'
print(sqlSelect)

## Switch data source names based on an env var: for testing only

In [ ]:
import os

# Comment out the set env code before scheduling it as a Job
# Set schema name for testing this notebook interactively

# Dev data source
#os.environ['CONN'] = 'DEV'
# Prod data source
#os.environ['CONN'] = 'PROD'

In [ ]:


connType=os.getenv('CONN')
print(connType)

In [ ]:
# Determine if this notebook needs to connect to a dev or a prod data source

# If connection wasn't set, the default connection is dev
if connType is None or connType == 'DEV' :
  connName = 'DB2_Dev'
  # For testing writing a file
  fileSuffix = '_Dev.csv'
elif connType == 'PROD':
  connName = 'DB2_Prod'
  fileSuffix = '_Prod.csv'
else:
  connName = 'DB2_Dev'
  # For testing writing a file
  fileSuffix = '_Dev.csv' 
  
  
print("Connection: " + connName)
print("Schema: " + schema_name)

In [ ]:
# Construct select string with schema_name variable
sqlSelect = 'SELECT * FROM ' + '"' + schema_name + '"."CUST_TRANS"'
print(sqlSelect)

In [ ]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

DB2_Prod_data_request = {
    'connection_name': connName,
    'select_statement': sqlSelect
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=DB2_Prod_data_request)

sampleData = itcfs.read_pandas_and_concat(readClient, flightInfo)
sampleData.head(10)

In [ ]:
# Write data frame to project - for testing

# Import the lib
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

filename = "SampleData" + fileSuffix
print(filename)

In [ ]:
# First, we need to write the file to working directory
sampleData.to_csv(filename)

# upload the intermediate file to your Watson Studio project
wslib.upload_file(file_path=filename, asset_name=filename)

*Written by Elena Lowery*
*May 2, 2022*